# 실습 퀴즈: 텐서 연산 완전 정복

## 사용 방법
- 각 문제 셀을 읽고, **직접 답을 예측한 후** 풀이 셀을 실행하세요
- 코드를 실행하기 전에 종이에 계산해보는 것을 권장합니다
- 틀렸다면 해설을 꼼꼼히 읽고, 왜 틀렸는지 이해하세요

## 목차
- [Q1: 행렬 곱 Shape 계산](#q1)
- [Q2: 브로드캐스팅 결과 Shape](#q2)
- [Q3: Transpose 결과 Shape](#q3)
- [Q4: GradientTape으로 미분 계산](#q4)
- [Q5: tf.Variable vs tf.constant](#q5)
- [종합 도전: 미니 신경망 구현](#bonus)

---

In [ ]:
# 퀴즈 환경 설정
import tensorflow as tf
import numpy as np

tf.random.set_seed(42)
np.random.seed(42)

print(f"TensorFlow 버전: {tf.__version__}")
print("퀴즈 환경 준비 완료!")
print("각 문제를 풀기 전에 먼저 답을 예측해보세요.")

---
## Q1: 행렬 곱 Shape 계산 <a name='q1'></a>

### 문제

다음 두 텐서 `A`와 `B`를 행렬 곱(`tf.matmul`)했을 때, 결과 텐서 `C`의 **shape**은?

```python
A = tf.zeros([3, 4])   # shape: (3, 4)
B = tf.zeros([4, 5])   # shape: (4, 5)
C = tf.matmul(A, B)
```

**힌트:** 행렬 곱 $C = AB$에서
- $A \in \mathbb{R}^{m \times k}$, $B \in \mathbb{R}^{k \times n}$이면
- $C \in \mathbb{R}^{m \times n}$
- 조건: $A$의 **열 수** = $B$의 **행 수**

**여러분의 예측:** C의 shape은 `(?, ?)` 입니다.

In [ ]:
# ── Q1 풀이 ──────────────────────────────────────────────────
A = tf.zeros([3, 4])   # (3행, 4열)
B = tf.zeros([4, 5])   # (4행, 5열)
C = tf.matmul(A, B)

print("=" * 45)
print("Q1 풀이: 행렬 곱 Shape")
print("=" * 45)
print(f"A.shape = {A.shape}  →  (m=3, k=4)")
print(f"B.shape = {B.shape}  →  (k=4, n=5)")
print(f"-" * 45)
print(f"C = A @ B")
print(f"C.shape = {C.shape}  →  (m=3, n=5)")
print()

# 수학적 설명
print("[해설]")
print("  행렬 곱 C_ij = Σ_k A_ik * B_kj")
print(f"  A의 열 수 ({A.shape[1]}) = B의 행 수 ({B.shape[0]}) → 곱 가능")
print(f"  결과 행 수 = A의 행 수 = {A.shape[0]}")
print(f"  결과 열 수 = B의 열 수 = {B.shape[1]}")
print(f"  ∴ C.shape = ({A.shape[0]}, {B.shape[1]})")

# 오류 케이스도 확인
print()
print("[오류 케이스] A(3,4) @ C(5,4) — 열/행 불일치")
try:
    D = tf.zeros([5, 4])   # 내부 차원 불일치
    E = tf.matmul(A, D)
except tf.errors.InvalidArgumentError as e:
    print(f"  예상된 오류 발생: {str(e)[:80]}...")

# 배치 행렬 곱 (3D)
print()
print("[심화] 배치 행렬 곱 (3D Tensor)")
A3 = tf.zeros([8, 3, 4])   # 배치 크기 8
B3 = tf.zeros([8, 4, 5])
C3 = tf.matmul(A3, B3)
print(f"  A3: {A3.shape}, B3: {B3.shape}")
print(f"  C3 = A3 @ B3: {C3.shape}  (배치 차원 유지)")

---
## Q2: 브로드캐스팅 결과 Shape <a name='q2'></a>

### 문제

다음 두 텐서를 더할 때 결과 shape은?

```python
A = tf.zeros([1, 3])   # shape: (1, 3)
B = tf.zeros([4, 3])   # shape: (4, 3)
C = A + B
```

**브로드캐스팅 규칙 복습:**
1. 오른쪽부터 차원을 비교
2. 차원 크기가 같으면 그대로
3. 한쪽이 **1**이면 다른 쪽 크기로 확장
4. 한쪽이 더 낮은 rank라면 왼쪽에 1을 추가

**여러분의 예측:** C의 shape은 `(?, ?)` 입니다.

---

**추가 문제:** 다음 중 브로드캐스팅이 **불가능**한 경우는?
- (a) `(2, 3) + (1, 3)`
- (b) `(3, 1) + (1, 4)` 
- (c) `(2, 3) + (4, 3)` ← 이것은?
- (d) `(1,) + (3, 4)`

In [ ]:
# ── Q2 풀이 ──────────────────────────────────────────────────
print("=" * 50)
print("Q2 풀이: 브로드캐스팅 결과 Shape")
print("=" * 50)

# 기본 문제
A = tf.zeros([1, 3])
B = tf.zeros([4, 3])
C = A + B

print(f"A.shape = {A.shape}  (1행, 3열)")
print(f"B.shape = {B.shape}  (4행, 3열)")
print(f"C = A + B → C.shape = {C.shape}")
print()
print("[해설]")
print("  열 차원 비교: 3 == 3  → 그대로 (3)")
print("  행 차원 비교: 1 vs 4  → 1이므로 4로 확장")
print("  A가 4번 복사되어 더해지는 효과")
print(f"  ∴ 결과 shape = (4, 3)")

# 실제 값으로 확인
A_val = tf.constant([[10, 20, 30]])   # (1,3)
B_val = tf.constant([[1, 2, 3],
                     [4, 5, 6],
                     [7, 8, 9],
                     [10,11,12]])     # (4,3)
C_val = A_val + B_val
print()
print("실제 값 확인:")
print(f"  A_val = {A_val.numpy()}")
print(f"  B_val =\n{B_val.numpy()}")
print(f"  C_val = A + B =\n{C_val.numpy()}")
print("  → A의 [10,20,30]이 각 행에 더해짐")

print()
print("=" * 50)
print("추가 문제: 브로드캐스팅 가능/불가능 케이스")
print("=" * 50)

cases = [
    ([2, 3], [1, 3], "(a)"),
    ([3, 1], [1, 4], "(b)"),
    ([2, 3], [4, 3], "(c) 불가능?"),
    ([1],    [3, 4], "(d)"),
]

for shape1, shape2, label in cases:
    try:
        t1 = tf.zeros(shape1)
        t2 = tf.zeros(shape2)
        result = t1 + t2
        print(f"  {label}: {tuple(shape1)} + {tuple(shape2)} → {tuple(result.shape.as_list())} [가능]")
    except (tf.errors.InvalidArgumentError, Exception) as e:
        print(f"  {label}: {tuple(shape1)} + {tuple(shape2)} → 오류! [불가능] ({str(e)[:50]})")

print()
print("[핵심] (c) (2,3)+(4,3): 2≠4이고 둘 다 1이 아님 → 브로드캐스팅 불가!")

In [ ]:
# Q2 심화: 다양한 브로드캐스팅 패턴

print("브로드캐스팅 심화 패턴:")

# (3,) + (2,3) → (2,3): 낮은 rank에 왼쪽 1 추가
v = tf.constant([1, 2, 3], dtype=tf.float32)          # (3,) → 사실상 (1,3)
m = tf.constant([[10,20,30],[40,50,60]], dtype=tf.float32)  # (2,3)
r = v + m
print(f"\n  (3,) + (2,3): {v.shape} + {m.shape} → {r.shape}")
print(f"  결과:\n{r.numpy()}")

# (3,1) + (1,4) → (3,4): 외적 덧셈
col = tf.constant([[1],[2],[3]], dtype=tf.float32)        # (3,1)
row = tf.constant([[10,20,30,40]], dtype=tf.float32)      # (1,4)
outer_sum = col + row
print(f"\n  (3,1) + (1,4): {col.shape} + {row.shape} → {outer_sum.shape}  (외적 덧셈)")
print(f"  결과:\n{outer_sum.numpy()}")

# 스칼라 브로드캐스팅
big_tensor = tf.zeros([3, 4, 5])
scalar = tf.constant(7.0)  # shape ()
r2 = big_tensor + scalar
print(f"\n  () + (3,4,5): {scalar.shape} + {big_tensor.shape} → {r2.shape}  (스칼라는 모든 원소에 적용)")

---
## Q3: Transpose 결과 Shape <a name='q3'></a>

### 문제

다음 3D 텐서에 `transpose`를 적용했을 때 결과 shape은?

```python
T = tf.zeros([2, 3, 4])   # shape: (2, 3, 4)
T_perm = tf.transpose(T, perm=[2, 0, 1])
```

**힌트:** `perm=[2, 0, 1]`이 의미하는 것:
- 새로운 0번 축 ← 기존 **2번** 축 (크기: 4)
- 새로운 1번 축 ← 기존 **0번** 축 (크기: 2)
- 새로운 2번 축 ← 기존 **1번** 축 (크기: 3)

**여러분의 예측:** T_perm의 shape은 `(?, ?, ?)` 입니다.

In [ ]:
# ── Q3 풀이 ──────────────────────────────────────────────────
print("=" * 50)
print("Q3 풀이: Transpose 결과 Shape")
print("=" * 50)

T = tf.zeros([2, 3, 4])
T_perm = tf.transpose(T, perm=[2, 0, 1])

print(f"원본 T.shape = {T.shape}")
print(f"             축0=2, 축1=3, 축2=4")
print()
print(f"perm = [2, 0, 1]")
print(f"  새 축0 ← 기존 축2  (크기: {T.shape[2]})")
print(f"  새 축1 ← 기존 축0  (크기: {T.shape[0]})")
print(f"  새 축2 ← 기존 축1  (크기: {T.shape[1]})")
print()
print(f"T_perm.shape = {T_perm.shape}")
print()

# 실제 데이터로 확인
T_val = tf.reshape(tf.range(24), [2, 3, 4])
T_perm_val = tf.transpose(T_val, perm=[2, 0, 1])

print("[해설]")
print(f"  원본 shape: (2, 3, 4)")
print(f"  perm[2, 0, 1]: 기존 [축2, 축0, 축1] = [4, 2, 3]")
print(f"  ∴ 결과 shape = (4, 2, 3)")

print()
print("다양한 perm 예시:")
perms = {
    '[0,1,2]': [0,1,2],   # 변화 없음
    '[1,0,2]': [1,0,2],   # 처음 두 축 교환
    '[2,1,0]': [2,1,0],   # 완전 역순
    '[2,0,1]': [2,0,1],   # 문제의 perm
    '[1,2,0]': [1,2,0],   # 순환 이동
}

for perm_name, perm in perms.items():
    result = tf.transpose(T, perm=perm)
    print(f"  perm={perm_name}: (2,3,4) → {tuple(result.shape.as_list())}")

In [ ]:
# Q3 심화: Transpose 원소 위치 변화 추적

# 작은 텐서로 원소 위치가 어떻게 바뀌는지 확인
small = tf.reshape(tf.range(12), [2, 3, 2])  # shape (2,3,2)
print(f"원본 shape: {small.shape}")
print(f"원본 값:")
print(small.numpy())

# 기본 transpose (역순: perm=[1,0,2] → 처음 두 축 교환)
t1 = tf.transpose(small, perm=[1, 0, 2])  # (3,2,2)
print(f"\ntranspose(perm=[1,0,2]) → shape: {t1.shape}")
print(t1.numpy())

print(f"\n[검증] 원본 small[0,1,0] = {small[0,1,0].numpy()}")
print(f"       변환 후  t1[1,0,0] = {t1[1,0,0].numpy()}")
print("       perm=[1,0,2] 이므로 축0↔축1 교환, 즉 [i,j,k] → [j,i,k]")

---
## Q4: GradientTape으로 미분 계산 <a name='q4'></a>

### 문제

`tf.GradientTape`을 사용하여 $f(x) = 3x^2 + 2x + 1$의 $x = 2$에서의 기울기(미분값)를 구하세요.

**수학적 풀이:**
$$f(x) = 3x^2 + 2x + 1$$
$$f'(x) = 6x + 2$$
$$f'(2) = 6 \times 2 + 2 = 14$$

**빈칸을 채우세요:**
```python
x = tf.Variable(___)

with tf.GradientTape() as tape:
    f = ___ * x**2 + ___ * x + ___

df_dx = tape.gradient(___, ___)
print(df_dx.numpy())  # 예상 결과: 14.0
```

In [ ]:
# ── Q4 풀이 ──────────────────────────────────────────────────
print("=" * 50)
print("Q4 풀이: GradientTape 미분 계산")
print("=" * 50)

# f(x) = 3x^2 + 2x + 1
x = tf.Variable(2.0)   # x = 2

with tf.GradientTape() as tape:
    f = 3 * x**2 + 2 * x + 1

df_dx = tape.gradient(f, x)

print(f"f(x) = 3x² + 2x + 1")
print(f"x = {x.numpy()}")
print(f"f(2) = 3×{x.numpy()}² + 2×{x.numpy()} + 1 = {f.numpy()}")
print(f"      (= 3×4 + 4 + 1 = 12 + 4 + 1 = 17)")
print()
print(f"f'(x) = 6x + 2")
print(f"f'(2) = 6×{x.numpy()} + 2 = {6*2+2}")
print()
print(f"GradientTape 결과: df_dx = {df_dx.numpy()}")
print(f"수학적 정답:         f'(2) = 14")
print(f"일치 여부: {abs(df_dx.numpy() - 14.0) < 1e-5}")

In [ ]:
# Q4 심화: 여러 x값에서 미분값 계산 및 시각화

# f(x) = 3x^2 + 2x + 1, f'(x) = 6x + 2

def compute_gradient(x_val):
    """주어진 x값에서 f(x)와 f'(x)를 반환"""
    x = tf.Variable(float(x_val))
    with tf.GradientTape() as tape:
        f = 3 * x**2 + 2 * x + 1
    df_dx = tape.gradient(f, x)
    return f.numpy(), df_dx.numpy()

print("f(x) = 3x² + 2x + 1  |  f'(x) = 6x + 2")
print("-" * 55)
print(f"{'x':>5} | {'f(x)':>10} | {'f\'(x) TF':>12} | {'f\'(x) 수식':>12}")
print("-" * 55)

for x_val in [-3, -2, -1, 0, 1, 2, 3]:
    f_val, grad_val = compute_gradient(x_val)
    theoretical = 6 * x_val + 2
    print(f"{x_val:>5} | {f_val:>10.1f} | {grad_val:>12.1f} | {theoretical:>12.1f}")

print("-" * 55)
print()
print("[해설] f'(x) = 6x + 2 = 0 → x = -1/3 ≈ -0.333 에서 최솟값")
f_min, _ = compute_gradient(-1/3)
print(f"       f(-1/3) ≈ {f_min:.4f}  (최솟값)")

In [ ]:
# Q4 응용: 경사하강법으로 f(x)의 최솟값 찾기
# GradientTape + 경사하강법 = 딥러닝 학습의 핵심 패턴

print("경사하강법으로 f(x) = 3x² + 2x + 1의 최솟값 찾기")
print(f"이론적 최솟값: x = -1/3 ≈ {-1/3:.4f}")
print()

x = tf.Variable(3.0)   # 임의의 시작점
lr = 0.05              # 학습률

print(f"{'에포크':>6} | {'x':>10} | {'f(x)':>10} | {'f\'(x)':>10}")
print("-" * 45)

for epoch in range(31):
    with tf.GradientTape() as tape:
        f = 3 * x**2 + 2 * x + 1
    grad = tape.gradient(f, x)
    x.assign_sub(lr * grad)   # x ← x - lr * f'(x)
    
    if epoch % 5 == 0:
        print(f"{epoch:>6} | {x.numpy():>10.6f} | {f.numpy():>10.6f} | {grad.numpy():>10.6f}")

print("-" * 45)
print(f"\n수렴된 x = {x.numpy():.6f}")
print(f"이론적 x = {-1/3:.6f}")
print(f"오차 = {abs(x.numpy() - (-1/3)):.8f}")

---
## Q5: tf.Variable vs tf.constant <a name='q5'></a>

### 문제

신경망의 학습 가능한 파라미터를 만들 때 `tf.Variable`과 `tf.constant` 중 어떤 것을 사용해야 할까요?

다음 코드 중 **학습이 올바르게 동작하는** 버전은?

```python
# 버전 A
w = tf.constant([1.0, 2.0, 3.0])
with tf.GradientTape() as tape:
    y = tf.reduce_sum(w * x)
grad = tape.gradient(y, w)  # grad는?

# 버전 B
w = tf.Variable([1.0, 2.0, 3.0])
with tf.GradientTape() as tape:
    y = tf.reduce_sum(w * x)
grad = tape.gradient(y, w)  # grad는?
```

**예측:** 어떤 버전이 None이 아닌 그래디언트를 반환할까요?

In [ ]:
# ── Q5 풀이 ──────────────────────────────────────────────────
print("=" * 55)
print("Q5 풀이: tf.Variable vs tf.constant")
print("=" * 55)

x = tf.constant([1.0, 2.0, 3.0])

# 버전 A: tf.constant 사용
w_const = tf.constant([1.0, 2.0, 3.0])
with tf.GradientTape() as tape:
    y_a = tf.reduce_sum(w_const * x)
grad_a = tape.gradient(y_a, w_const)

print(f"[버전 A] w = tf.constant")
print(f"  그래디언트 결과: {grad_a}")
print(f"  → None! GradientTape은 Variable만 자동 추적함")

print()

# 버전 B: tf.Variable 사용
w_var = tf.Variable([1.0, 2.0, 3.0])
with tf.GradientTape() as tape:
    y_b = tf.reduce_sum(w_var * x)
grad_b = tape.gradient(y_b, w_var)

print(f"[버전 B] w = tf.Variable")
print(f"  그래디언트 결과: {grad_b.numpy()}")
print(f"  → 정상! y = w1*x1 + w2*x2 + w3*x3")
print(f"  → ∂y/∂w = [x1, x2, x3] = {x.numpy()}")
print()

# 예외: constant도 tape.watch()로 추적 가능
w_const2 = tf.constant([1.0, 2.0, 3.0])
with tf.GradientTape() as tape:
    tape.watch(w_const2)   # 명시적 추적
    y_c = tf.reduce_sum(w_const2 * x)
grad_c = tape.gradient(y_c, w_const2)

print(f"[예외] tf.constant + tape.watch()")
print(f"  그래디언트 결과: {grad_c.numpy()}")
print(f"  → tape.watch()로 명시적 추적 시 가능")

In [ ]:
# Q5 심화: 학습 가능한 파라미터 직접 구현
# trainable=False로 특정 파라미터의 학습을 중지할 수 있음

print("학습 가능/불가능 파라미터 제어:")
print()

# 학습 가능 (기본)
w_trainable = tf.Variable([1.0, 2.0], name='trainable_weight', trainable=True)
# 학습 불가 (예: 고정된 임베딩, BatchNorm 통계)
w_frozen    = tf.Variable([3.0, 4.0], name='frozen_weight', trainable=False)

print(f"w_trainable.trainable = {w_trainable.trainable}")
print(f"w_frozen.trainable    = {w_frozen.trainable}")

x_input = tf.constant([1.0, 1.0])

with tf.GradientTape() as tape:
    # trainable=False인 Variable은 기본적으로 추적 안 됨
    output = tf.reduce_sum(w_trainable * x_input) + tf.reduce_sum(w_frozen * x_input)

grad_trainable = tape.gradient(output, w_trainable)
grad_frozen    = tape.gradient(output, w_frozen)   # 주의: 이미 tape 소비됨

print(f"\n그래디언트 결과:")
print(f"  ∂output/∂w_trainable = {grad_trainable.numpy() if grad_trainable is not None else 'None'}")
print(f"  ∂output/∂w_frozen    = {grad_frozen}  (trainable=False이므로 추적 안됨)")

print()

# Keras 레이어에서 trainable 제어
layer = tf.keras.layers.Dense(4, name='my_layer')
_ = layer(tf.ones([1, 3]))  # 레이어 빌드

print(f"Dense 레이어 파라미터:")
for var in layer.trainable_variables:
    print(f"  학습 가능 - {var.name}: shape={var.shape}")

# 레이어 동결
layer.trainable = False
print(f"\nlayer.trainable = False 후:")
print(f"  trainable_variables 수: {len(layer.trainable_variables)}  (0이 되어야 함)")
print(f"  non_trainable_variables 수: {len(layer.non_trainable_variables)}")

In [ ]:
# Q5 최종: tf.Variable의 주요 메서드 총정리

print("tf.Variable 주요 메서드 및 속성 총정리:")
print()

v = tf.Variable([[1.0, 2.0], [3.0, 4.0]], name='demo', trainable=True)

print(f"생성: v = tf.Variable([[1,2],[3,4]])")
print(f"  v.numpy()      = \n{v.numpy()}")
print(f"  v.name         = {v.name}")
print(f"  v.shape        = {v.shape}")
print(f"  v.dtype        = {v.dtype}")
print(f"  v.trainable    = {v.trainable}")
print()

print("값 수정 메서드:")
v.assign([[10., 20.], [30., 40.]])
print(f"  v.assign([[10,20],[30,40]]):")
print(f"    {v.numpy()}")

v.assign_add([[1., 1.], [1., 1.]])
print(f"  v.assign_add([[1,1],[1,1]]):")
print(f"    {v.numpy()}")

v.assign_sub([[1., 1.], [1., 1.]])
print(f"  v.assign_sub([[1,1],[1,1]]):")
print(f"    {v.numpy()}")

# 특정 인덱스 업데이트
v[0, 1].assign(99.)
print(f"  v[0,1].assign(99.):")
print(f"    {v.numpy()}")

---
## 종합 도전: 미니 신경망 구현 <a name='bonus'></a>

지금까지 배운 내용을 종합하여, XOR 문제를 푸는 2층 신경망을 구현하세요.

**XOR 진리표:**
| x1 | x2 | y |
|----|----|----|  
| 0  | 0  | 0  |
| 0  | 1  | 1  |
| 1  | 0  | 1  |
| 1  | 1  | 0  |

**모델 구조:** Input(2) → Hidden(4, ReLU) → Output(1, Sigmoid)

**공식:**
$$h = \text{ReLU}(W_1 x + b_1)$$
$$\hat{y} = \sigma(W_2 h + b_2)$$
$$L = -\frac{1}{n}\sum_i \left[ y_i \log \hat{y}_i + (1 - y_i) \log(1 - \hat{y}_i) \right]$$

In [ ]:
# ── 종합 도전 풀이 ────────────────────────────────────────────
print("XOR 신경망 — GradientTape으로 수동 구현")
print("=" * 50)

# XOR 데이터
X_xor = tf.constant([[0.,0.],[0.,1.],[1.,0.],[1.,1.]], dtype=tf.float32)
Y_xor = tf.constant([[0.],[1.],[1.],[0.]], dtype=tf.float32)

print("XOR 입력 데이터:")
for i in range(4):
    print(f"  x={X_xor[i].numpy()} → y={Y_xor[i].numpy()}")

# 모델 파라미터 초기화 (tf.Variable 사용!)
tf.random.set_seed(7)
W1 = tf.Variable(tf.random.normal([2, 4], stddev=0.5), name='W1')  # 입력→은닉
b1 = tf.Variable(tf.zeros([4]),                         name='b1')
W2 = tf.Variable(tf.random.normal([4, 1], stddev=0.5), name='W2')  # 은닉→출력
b2 = tf.Variable(tf.zeros([1]),                         name='b2')

params = [W1, b1, W2, b2]
print(f"\n파라미터 수: {sum(p.numpy().size for p in params)}개")

In [ ]:
# 순전파 함수 정의
def forward(X):
    """순전파: Input → ReLU Hidden → Sigmoid Output"""
    # 1층: h = ReLU(W1 @ X^T + b1)
    z1 = tf.matmul(X, W1) + b1        # (4,2)@(2,4) + (4,) = (4,4)
    h  = tf.nn.relu(z1)               # ReLU 활성화
    # 2층: y_hat = Sigmoid(W2 @ h^T + b2)
    z2 = tf.matmul(h, W2) + b2        # (4,4)@(4,1) + (1,) = (4,1)
    y_hat = tf.nn.sigmoid(z2)         # Sigmoid 활성화
    return y_hat

def bce_loss(y_true, y_pred):
    """이진 교차 엔트로피 손실"""
    eps = 1e-7
    y_pred = tf.clip_by_value(y_pred, eps, 1.0 - eps)  # 수치 안정성
    return -tf.reduce_mean(
        y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred)
    )

# 학습 루프
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)

loss_history = []
print(f"{'에포크':>8} | {'BCE 손실':>12} | {'예측 (반올림)':>20}")
print("-" * 50)

for epoch in range(2001):
    with tf.GradientTape() as tape:
        y_pred = forward(X_xor)
        loss = bce_loss(Y_xor, y_pred)
    
    grads = tape.gradient(loss, params)
    optimizer.apply_gradients(zip(grads, params))
    
    loss_history.append(loss.numpy())
    
    if epoch % 500 == 0:
        preds = tf.round(y_pred).numpy().flatten()
        print(f"{epoch:>8} | {loss.numpy():>12.6f} | {preds}")

print("-" * 50)
print(f"\n학습 완료!")
print(f"최종 손실: {loss_history[-1]:.6f}")

In [ ]:
# 최종 예측 결과 확인
print("XOR 신경망 최종 예측 결과:")
print("-" * 50)
print(f"{'입력 x':>12} | {'실제 y':>8} | {'예측 확률':>12} | {'예측 (반올림)':>12} | {'정답':>6}")
print("-" * 50)

final_preds = forward(X_xor)
all_correct = True

for i in range(4):
    x_i = X_xor[i].numpy()
    y_true_i = Y_xor[i].numpy()[0]
    y_prob_i = final_preds[i].numpy()[0]
    y_pred_i = round(y_prob_i)
    correct = y_pred_i == y_true_i
    if not correct:
        all_correct = False
    status = "O" if correct else "X"
    print(f"  {str(x_i):>10} | {y_true_i:>8.0f} | {y_prob_i:>12.6f} | {y_pred_i:>12.0f} | {status:>6}")

print("-" * 50)
print(f"\n결과: {'모든 경우 정답!' if all_correct else 'XOR 아직 미완성 (더 학습 필요)'}")
print()
print("[요약] 이 XOR 구현에서 사용한 개념들:")
print("  - tf.Variable: 학습 가능한 파라미터 W1, b1, W2, b2")
print("  - tf.matmul:   행렬 곱으로 레이어 계산")
print("  - GradientTape: 자동 미분으로 역전파 계산")
print("  - Adam optimizer: 그래디언트 기반 파라미터 업데이트")
print("  - 브로드캐스팅: + b 편향 덧셈에 자동 적용")

---
## 전체 요약 및 핵심 공식 정리

### Q1~Q5 핵심 정리

| 문제 | 핵심 개념 | 결론 |
|------|-----------|------|
| **Q1** | 행렬 곱 shape | $(m, k) \times (k, n) \to (m, n)$ — 내부 차원 제거 |
| **Q2** | 브로드캐스팅 | 크기 1인 차원만 확장 가능. 1도 아니고 다르면 오류 |
| **Q3** | Transpose perm | `perm[i]` = 새 i번 축이 기존 몇 번 축인지 |
| **Q4** | GradientTape | `tape.gradient(y, x)` = $\frac{\partial y}{\partial x}$ |
| **Q5** | Variable vs Constant | 학습 파라미터는 반드시 `tf.Variable`로 선언 |

### 자주 쓰는 공식 모음

$$\text{행렬 곱: } C_{ij} = \sum_k A_{ik}B_{kj}$$

$$\text{편미분: } \frac{\partial f}{\partial x_i} \approx \frac{f(\ldots, x_i + \epsilon, \ldots) - f(\ldots, x_i, \ldots)}{\epsilon}$$

$$\text{연쇄 법칙: } \frac{\partial L}{\partial w} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial w}$$

$$\text{경사하강법: } w \leftarrow w - \eta \frac{\partial L}{\partial w}$$

$$\text{MSE: } L = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2$$

$$\text{BCE: } L = -\frac{1}{n}\sum_i \left[ y_i \log \hat{y}_i + (1-y_i)\log(1-\hat{y}_i) \right]$$

---

### 다음 단계

Chapter 01의 기초를 완료했습니다. 다음 챕터들에서는:
- **Chapter 02:** Keras API로 신경망 구조화
- **Chapter 03:** 활성화 함수, 손실 함수, 옵티마이저 심화
- **Chapter 04:** CNN, RNN 등 특화 레이어

수고하셨습니다!